In [ ]:
# Load in the dataset
import pandas as pd
import nbimporter
from get_game_stats_data import get_game_stats_data_df
import numpy as np


def get_X_and_y(df):
    df = df[df['WL'].ne('')]
    y = df['WL'].astype(float)
    home_ids = df['TEAM_ID'].astype(int).tolist()
    X = df.drop(columns=['WL', 'GAME_DATE', 'TEAM_ID'])
    return X, y, home_ids

def plot_save_confusion_matrix(cm, title_prefix=None):
    cm = np.rot90(cm, 2)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Positive", "Negative"])
    disp.plot(cmap="Blues", values_format="d")
    title = f"{title_prefix or ''}Confusion Matrix"
    plt.title(title)
    plt.tight_layout()
    plt.grid(False)
    filename = f"ml_imgs/{title.lower().replace(' ', '_')}.png"
    plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, balanced_accuracy_score, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import joblib
import nbimporter
from feature_engineering import get_dropped_features, correlation_matrix
import matplotlib.pyplot as plt


def train_model(X, y, get_corr_matrix=False, save_conf_matrix=False):
    tscv = TimeSeriesSplit(n_splits=10)
    best_thresholds = []
    accuracies = []
    recalls = []
    precisions = []
    f1s = []
    cms = []
    
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train_full, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train_full, y_test = y.iloc[train_index].astype(int), y.iloc[test_index].astype(int)
        X_train_full, X_val, y_train_full, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, shuffle=False)
    
        fold_data = dict()
        fold_data["feature_names_pre_drop"] = X_train_full.columns.tolist()
        
        fold_data["mean"] = X_train_full.mean()
        X_train_full = X_train_full.fillna(fold_data["mean"])
        X_val = X_val.fillna(fold_data["mean"])
        X_test = X_test.fillna(fold_data["mean"])
        
        fold_data["std_dev"] = X_train_full.std().to_dict()
    
        scaler = StandardScaler()
        X_train_full = pd.DataFrame(scaler.fit_transform(X_train_full), columns=X_train_full.columns, index=X_train_full.index)
        X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
        fold_data["scaler"] = scaler
        
        threshold_mag=0.7
        if get_corr_matrix:
            correlation_matrix(
                X_train_full, threshold_mag=threshold_mag, k=0, plot_corr=False, xlabel="Features", ylabel="Features", title_append=f" for Pre-Fold {i+1}", save_corr=True
            )
        dropped_features = get_dropped_features(X_train_full, threshold_mag=threshold_mag)
        X_train_full = X_train_full.drop(dropped_features, axis=1)
        X_test = X_test.drop(dropped_features, axis=1)
        X_val = X_val.drop(dropped_features, axis=1)
        if get_corr_matrix:
            correlation_matrix(
                X_train_full, threshold_mag=threshold_mag, k=0, plot_corr=False, xlabel="Features", ylabel="Features", title_append=f" for Post-Fold {i+1}", save_corr=True
            )
        fold_data["dropped_features"] = dropped_features
        
        seed = 42
        estimators = [
            ('rf', RandomForestClassifier(random_state=seed)),
            ('xgb', XGBClassifier(random_state=seed, eval_metric='logloss')),
            ('svm', SVC(probability=True, random_state=seed)),
            ('knn', KNeighborsClassifier()),
        ]
    
        stacking_clf = StackingClassifier(
            estimators=estimators,
            final_estimator=LogisticRegression(),
            cv=2,
            passthrough=True
        )
    
        stacking_clf.fit(X_train_full, y_train_full)
        fold_data["model"] = stacking_clf
    
        probs = stacking_clf.predict_proba(X_val)[:, 1]
        best_thresh = 0.5
        best_metric = 0
        for threshold in np.arange(0.3, 0.7, 0.01):
            y_pred_thresh = (probs >= threshold).astype(int)
            bal_acc = balanced_accuracy_score(y_val, y_pred_thresh)
            if bal_acc > best_metric:
                best_metric = bal_acc
                best_thresh = threshold
    
        fold_data["best_threshold"] = best_thresh
        joblib.dump(fold_data, f'nba_model_fold_data/ensemble_fold_{i}.pkl')
        
        probs = stacking_clf.predict_proba(X_test)[:, 1]
        y_pred = (probs >= best_thresh).astype(int)
        acc = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
    
        accuracies.append(acc)
        recalls.append(recall)
        precisions.append(precision)
        f1s.append(f1)
        cms.append(cm)
        best_thresholds.append(best_thresh)
    
    print("="*100)
    print(f"Average-across-folds accuracy: {np.mean(accuracies):.4f}")
    print(f"Average-across-folds recall: {np.mean(recalls):.4f}")
    print(f"Average-across-folds precision: {np.mean(precisions):.4f}")
    print(f"Average-across-folds f1: {np.mean(f1s):.4f}")
    print(f"Average-across-folds best thresholds: {np.mean(best_thresholds):.4f}")
    print("="*100)
    print("Confusion matrices:")
    for i, cm in enumerate(cms):
        print(f"Fold {i}:\n{cm}")
        if save_conf_matrix:
            plot_save_confusion_matrix(cm, title_prefix=f"Fold {i+1} ")
    print("="*100)

In [ ]:
train_model(X, y, get_corr_matrix=False, save_conf_matrix=False)

In [31]:
season_year = "2024-25"
df = get_game_stats_data_df(season_year, training_and_testing=True)
X, y, home_ids = get_X_and_y(df)

get_game_stats_data.ipynb:107: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  "            avg_col, max_col = get_max_avg_col(col_list)\n",


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
import os
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=".*Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated.*"
)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)


def make_preds(X, y, verbose=False, get_conf_matrix_img=False, season_year="Test"):
    if X.shape[0] < 1:
        return None, None, None, None, None, None
    
    ensemble_probs = []
    ensemble_preds = []
    accs = []
    recalls = []
    pres = []
    f1s = []
    cms = []
    
    folder_path = 'nba_model_fold_data'
    num_folds = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    for i in range(num_folds):
        fold = joblib.load(f'nba_model_fold_data/ensemble_fold_{i}.pkl')
        X_test = X.copy()
        y_test = y.copy()
    
        X_test = X_test.reindex(columns=fold['feature_names_pre_drop'], fill_value=0)
        X_test = X_test.fillna(fold['mean'])
        X_test = pd.DataFrame(
            fold['scaler'].transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )
        X_test = X_test.drop(fold['dropped_features'], axis=1)
    
        probs = fold['model'].predict_proba(X_test)[:, 1]
        y_preds = (probs >= fold['best_threshold']).astype(int)
    
        ensemble_probs.append(probs)
        ensemble_preds.append(y_preds)
    
        acc = accuracy_score(y_test, y_preds)
        recall = recall_score(y_test, y_preds)
        precision = precision_score(y_test, y_preds)
        f1 = f1_score(y_test, y_preds)
        cm = confusion_matrix(y_test, y_preds, labels=[0, 1])
        accs.append(acc)
        recalls.append(recall)
        pres.append(precision)
        f1s.append(f1)
        cms.append(cm)
    
    ensemble_preds = [np.array(p).flatten() for p in ensemble_preds]
    preds_stack = np.vstack(ensemble_preds)
    final_preds = stats.mode(preds_stack, axis=0, keepdims=False).mode
    final_per = np.mean(preds_stack, axis=0)
    # print("final preds:", final_preds)
    ensemble_probs = [np.array(p).flatten() for p in ensemble_probs]
    probs_stack = np.vstack(ensemble_probs)
    # final_probs = stats.mode(preds_stack, axis=0, keepdims=False).mode
    
    
    final_acc = accuracy_score(y, final_preds)
    final_recall = recall_score(y, final_preds, zero_division=0)
    final_precision = precision_score(y, final_preds, zero_division=0)
    final_f1 = f1_score(y, final_preds, zero_division=0)
    final_cm = confusion_matrix(y, final_preds, labels=[0, 1])

    if verbose:
        print("-" * 100)
        print(f"Accuracy: {final_acc:.4f}")
        print(f"Recall: {final_recall:.4f}")
        print(f"Precision: {final_precision:.4f}")
        print(f"F1 Score: {final_f1:.4f}")
        print("Confusion matrix:")
        print(final_cm)
        print("-" * 100)
        
    if get_conf_matrix_img:
        plot_save_confusion_matrix(final_cm, title_prefix=f"{season_year} Season ")

    return list(final_preds), float(final_per[0]), final_acc, final_recall, final_precision, final_f1, final_cm


def smooth_clip(avg, max_val, alpha=2):
    smooth = 0.95 * max_val * np.tanh(alpha * avg / max_val) / np.tanh(alpha)
    return np.minimum(avg, smooth)


def CI95_percentage(x, n):
    lower_bound = np.round(x - 1.96 * np.sqrt(x * (100 - x)/n), 2)
    upper_bound = np.round(x + 1.96 * np.sqrt(x * (100 - x)/n), 2)
    return [lower_bound, upper_bound]



def get_sim_pred(X, row_idx, folds, scale_factor, pred_threshold, num_sims=1000):
    X_row = X.iloc[[row_idx]]
    ensemble_preds = []

    for i in range(len(folds)):
        fold = folds[i]
        X_fold = X_row.reindex(columns=fold['feature_names_pre_drop'], fill_value=0)
        X_fold.fillna(fold['mean'], inplace=True)

        X_sim = np.tile(X_fold.values, (num_sims, 1))
        X_sim = pd.DataFrame(X_sim, columns=X_fold.columns)

        std_devs = fold['std_dev']
        np.random.seed(42)
        for feature, std_dev in std_devs.items():
            noise = np.random.normal(loc=0, scale=scale_factor*std_dev, size=num_sims)
            X_sim[feature] += noise

            if "MAX" in feature and "PCT" not in feature:
                X_sim[feature] = np.round(X_sim[feature])
            elif "PCT" in feature:
                X_sim[feature] = np.clip(X_sim[feature], 0, 1)
            elif "AVG" in feature and "PLUS_MINUS" not in feature:
                X_sim[feature] = np.clip(X_sim[feature], 0, None)

        avg_max_pairs = [
            (feature, feature.replace("AVG", "MAX"))
            for feature in X_sim.columns if "AVG" in feature
        ]
        for avg_feature, max_feature in avg_max_pairs:
            if max_feature in X_sim.columns:
                X_sim[avg_feature] = smooth_clip(X_sim[avg_feature], X_sim[max_feature])
        
        X_sim.fillna(fold['mean'], inplace=True)
        X_sim = pd.DataFrame(
            fold['scaler'].transform(X_sim),
            columns=X_sim.columns,
            index=X_sim.index
        )
        X_sim.drop(columns=fold['dropped_features'], inplace=True)

        probs = fold['model'].predict_proba(X_sim)[:, 1]
        y_preds = (probs >= fold['best_threshold']).astype(int)
        ensemble_preds.append(y_preds)

    preds_stack = np.vstack(ensemble_preds)
    final_preds = preds_stack.mean(axis=0)

    final_pred = int(np.mean(final_preds) >= pred_threshold)
    number_wins = final_preds.sum()
    percentage = np.round(number_wins / num_sims * 100, 2)

    # print(f"Simulation prediction: {final_pred}")
    # print(f"Home team (1) won in {percentage}% of the {num_sims} simulations")

    return float(final_pred), percentage

def get_all_preds(X, y, scale_factor, pred_threshold, num_sims=1000):
    # num_test_cases = 10
    num_test_cases = X.shape[0]
    print(num_test_cases)
    print(f"pred_threshold: {pred_threshold}")
    print(f"scale_factor: {scale_factor}")
    X_test = X.iloc[:num_test_cases]
    y_test = y.iloc[:num_test_cases]

    det_preds = [None] * num_test_cases
    det_pers = [None] * num_test_cases
    sim_preds = [None] * num_test_cases
    sim_pers = [None] * num_test_cases
    
    folder_path = 'nba_model_fold_data'
    fold_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    num_folds = len(fold_files)
    folds = [joblib.load(f'{folder_path}/ensemble_fold_{i}.pkl') for i in range(num_folds)]
    for i in range(num_test_cases):
        # print("=" * 50)
        # print(f"Game {i+1} of the Season")
        # print("-" * 30)

        x_i = X.iloc[[i]]
        y_i = y.iloc[i]

        det_pred, det_per, *_ = make_preds(
            x_i, np.array([[y_i]]),
            verbose=False, 
            get_conf_matrix_img=False, 
            season_year=season_year
        )

        # print(f"Deterministic prediction: {det_pred[0]}")
        # print(f"Deterministic percentage: {det_per}")

        sim_pred, percentage = get_sim_pred(X_test, i, folds, scale_factor, pred_threshold, num_sims=num_sims)
        sim_pred = int(np.round(sim_pred))

        # print(f"Actual outcome: {y_i}")
        CI = CI95_percentage(percentage, num_sims)
        # print(f"We are 95% confident that the interval from {CI[0]}% to {CI[1]}% (both inclusive) contains the true simulation win percentage for the home team")
        # print("=" * 50)

        det_preds[i] = det_pred[0]
        det_pers[i] = det_per
        sim_preds[i] = sim_pred
        sim_pers[i] = percentage

    print(f"y test:      {list(y_test)}")
    print(f"det preds:   {det_preds}")
    print(f"det pers:    {det_pers}")
    print(f"sim preds:   {sim_preds}")
    print(f"sim pers:    {sim_pers}")

    det_acc = accuracy_score(y_test, det_preds) * 100
    sim_acc = accuracy_score(y_test, sim_preds) * 100

    det_recall = recall_score(y_test, det_preds)
    sim_recall = recall_score(y_test, sim_preds)

    det_precision = precision_score(y_test, det_preds)
    sim_precision = precision_score(y_test, sim_preds)

    det_f1 = f1_score(y_test, det_preds)
    sim_f1 = f1_score(y_test, sim_preds)

    det_cm = confusion_matrix(y_test, det_preds, labels=[0, 1])
    sim_cm = confusion_matrix(y_test, sim_preds, labels=[0, 1])

    print("=" * 50)
    print("Deterministic vs. Simulation Performance:")
    print("-" * 30)
    print(f"Deterministic accuracy: {det_acc:.2f}%")
    print(f"Simulation accuracy:    {sim_acc:.2f}%")
    print(f"Deterministic recall:   {det_recall:.2f}")
    print(f"Simulation recall:      {sim_recall:.2f}")
    print(f"Deterministic precision:{det_precision:.2f}")
    print(f"Simulation precision:   {sim_precision:.2f}")
    print(f"Deterministic f1:       {det_f1:.2f}")
    print(f"Simulation f1:          {sim_f1:.2f}")
    print(f"Deterministic cm:\n{det_cm}")
    print(f"Simulation cm:\n{sim_cm}")
    print("=" * 50)

    return X_test, y_test, det_preds, det_pers, sim_preds, sim_pers

In [141]:
scale_factors = [0.25, 0.5]
pred_thresholds = [0.5, 0.55, 0.6, 0.65, 0.7]

results = {}
for pred_threshold in pred_thresholds:
    results[pred_threshold] = {}
    for scale_factor in scale_factors:
        results[pred_threshold][scale_factor] = {}
        X_test, y_test, det_preds, det_pers, sim_preds, sim_pers = get_all_preds(
            X, y, scale_factor, pred_threshold, num_sims=1000
        )
        results[pred_threshold][scale_factor]["X_test"] = X_test
        results[pred_threshold][scale_factor]["y_test"] = y_test
        results[pred_threshold][scale_factor]["det_preds"] = det_preds
        results[pred_threshold][scale_factor]["det_pers"] = det_pers
        results[pred_threshold][scale_factor]["sim_preds"] = sim_preds
        results[pred_threshold][scale_factor]["sim_pers"] = sim_pers

y test:      [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,

KeyboardInterrupt: 

In [152]:
y_test = [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
det_preds = [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1]
det_pers = [0.9, 0.8, 0.9, 0.8, 0.9, 0.0, 0.2, 1.0, 0.3, 1.0, 0.1, 0.1, 0.8, 0.0, 0.4, 0.9, 0.9, 0.1, 0.9, 0.7, 0.9, 0.2, 0.9, 1.0, 0.1, 0.1, 0.9, 0.9, 0.9, 0.8, 0.1, 0.9, 0.5, 0.9, 0.5, 0.9, 0.9, 0.4, 0.3, 0.9, 0.9, 0.9, 0.9, 0.4, 0.6, 0.8, 0.9, 0.9, 0.9, 0.2, 0.6, 0.0, 0.9, 0.5, 0.5, 0.5, 0.9, 0.1, 0.4, 0.8, 0.9, 0.6, 0.1, 0.0, 0.3, 0.8, 0.5, 0.9, 0.9, 0.9, 0.1, 0.8, 0.4, 0.9, 0.6, 0.9, 0.9, 0.9, 0.3, 1.0, 0.9, 0.9, 0.9, 0.9, 0.1, 0.9, 0.6, 0.0, 0.1, 0.9, 0.9, 0.3, 0.9, 0.3, 0.9, 0.9, 0.1, 0.9, 0.7, 0.9, 0.2, 0.9, 0.5, 0.4, 0.1, 0.7, 0.9, 0.4, 0.9, 0.7, 0.9, 0.8, 0.9, 0.7, 1.0, 0.5, 0.9, 0.6, 0.9, 0.9, 0.4, 0.9, 0.9, 0.9, 0.9, 0.2, 0.9, 0.8, 0.9, 0.6, 0.6, 0.9, 0.4, 0.5, 0.4, 0.8, 0.2, 0.9, 0.2, 0.9, 0.1, 0.9, 0.2, 0.9, 0.4, 0.4, 0.1, 0.2, 0.6, 0.9, 0.4, 0.8, 0.3, 0.9, 0.5, 0.9, 0.3, 0.5, 0.2, 0.2, 0.0, 0.7, 0.6, 0.9, 0.8, 0.9, 0.4, 1.0, 0.3, 0.7, 0.9, 0.6, 0.0, 0.2, 0.9, 0.2, 0.9, 0.3, 0.9, 0.9, 0.3, 0.2, 0.2, 0.9, 0.4, 0.2, 0.3, 0.6, 0.9, 0.9, 1.0, 0.5, 0.3, 0.8, 0.6, 0.9, 0.9, 0.9, 0.5, 0.9, 0.9, 0.8, 1.0, 0.4, 0.1, 0.8, 0.9, 0.4, 0.2, 0.3, 0.9, 0.6, 0.2, 0.8, 0.3, 0.9, 0.5, 0.8, 0.3, 0.9, 0.2, 0.3, 0.1, 0.9, 0.4, 0.9, 0.9, 0.9, 0.9, 0.9, 0.2, 0.4, 0.9, 0.5, 0.9, 0.3, 0.9, 0.3, 0.9, 0.9, 0.9, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.9, 0.3, 0.9, 0.5, 0.2, 0.9, 0.9, 0.4, 0.2, 0.3, 0.3, 0.9, 0.8, 0.2, 0.2, 0.9, 0.7, 0.3, 0.8, 0.5, 0.3, 0.3, 0.9, 0.5, 0.1, 0.9, 0.9, 0.9, 0.2, 0.2, 0.3, 0.9, 0.9, 0.5, 0.2, 0.1, 0.5, 0.5, 0.9, 0.9, 0.8, 1.0, 0.9, 0.9, 0.3, 0.2, 0.4, 0.9, 0.2, 0.9, 0.8, 0.9, 0.4, 0.9, 0.6, 0.9, 0.3, 0.5, 0.8, 0.9, 0.3, 0.3, 0.9, 0.1, 0.9, 0.2, 0.7, 0.5, 0.3, 0.5, 0.8, 0.4, 0.9, 0.3, 0.9, 0.3, 0.3, 0.2, 0.9, 0.3, 0.6, 0.9, 0.8, 0.8, 0.9, 0.5, 0.3, 0.1, 0.2, 0.9, 0.7, 0.9, 0.7, 0.6, 0.2, 0.4, 0.9, 0.9, 0.9, 0.8, 0.9, 0.2, 0.4, 0.9, 0.9, 0.2, 0.9, 0.8, 0.9, 1.0, 0.9, 0.3, 0.2, 0.9, 0.0, 0.3, 0.2, 0.2, 0.2, 0.2, 0.3, 0.5, 0.8, 0.8, 0.2, 0.4, 0.9, 0.9, 0.9, 0.3, 0.6, 0.3, 0.5, 0.2, 0.6, 0.9, 0.8, 0.8, 0.8, 0.3, 0.0, 0.4, 0.3, 0.9, 0.7, 0.1, 0.9, 0.9, 0.4, 0.9, 0.9, 0.4, 0.5, 0.2, 0.9, 0.4, 0.9, 0.9, 0.9, 0.2, 0.2, 0.9, 0.6, 0.2, 0.2, 0.8, 0.2, 0.9, 0.5, 0.9, 0.3, 0.9, 0.9, 0.4, 0.3, 0.9, 0.0, 0.6, 0.5, 0.9, 0.2, 0.9, 0.0, 0.9, 0.2, 0.6, 0.1, 0.5, 0.2, 0.9, 0.0, 0.9, 0.0, 0.7, 0.0, 0.2, 0.2, 0.9, 0.5, 0.9, 0.5, 0.3, 0.6, 0.1, 0.2, 0.7, 0.3, 1.0, 0.9, 0.9, 0.3, 0.5, 0.5, 0.5, 0.8, 0.3, 0.4, 0.8, 0.3, 0.2, 0.1, 0.2, 0.3, 0.3, 0.5, 0.9, 0.9, 0.6, 0.9, 0.2, 0.3, 0.5, 0.7, 0.5, 0.2, 0.9, 0.1, 0.8, 0.5, 0.9, 0.8, 0.5, 0.2, 0.9, 0.6, 0.8, 0.8, 0.5, 0.2, 0.0, 0.9, 0.0, 0.9, 0.9, 0.9, 0.8, 0.7, 0.9, 0.3, 0.7, 0.0, 0.2, 0.9, 0.0, 0.0, 0.2, 0.9, 0.3, 0.2, 0.6, 0.2, 0.3, 0.9, 0.6, 0.4, 0.9, 0.2, 0.8, 0.0, 0.8, 0.3, 0.4, 0.9, 0.2, 0.5, 0.0, 0.6, 0.1, 0.1, 0.3, 0.8, 0.8, 0.7, 0.9, 0.8, 0.0, 0.6, 0.5, 0.9, 0.4, 0.3, 0.7, 0.0, 0.9, 0.9, 0.2, 0.5, 0.9, 0.9, 0.4, 0.9, 0.0, 0.7, 0.8, 0.9, 0.3, 0.2, 0.9, 0.8, 0.1, 0.0, 0.8, 0.3, 0.3, 0.2, 0.3, 0.6, 0.4, 0.6, 0.8, 0.4, 0.1, 0.9, 0.1, 0.9, 0.9, 0.9, 0.9, 0.2, 0.9, 0.6, 0.6, 0.9, 0.0, 0.3, 0.7, 0.2, 0.9, 0.4, 0.9, 0.8, 0.2, 0.7, 0.1, 0.9, 0.2, 0.8, 0.9, 0.9, 0.9, 0.9, 0.2, 0.2, 0.3, 0.7, 0.0, 0.8, 0.1, 0.9, 0.4, 0.8, 0.7, 0.0, 0.8, 0.1, 0.3, 0.7, 0.8, 0.6, 0.7, 0.6, 0.2, 0.9, 0.9, 0.3, 0.2, 1.0, 0.9, 0.9, 0.9, 0.7, 0.8, 0.0, 0.2, 0.9, 0.4, 0.1, 0.8, 0.3, 0.8, 0.6, 0.2, 0.2, 0.6, 0.6, 0.7, 0.2, 0.0, 0.4, 0.3, 0.6, 0.3, 0.9, 0.8, 0.8, 0.3, 0.9, 1.0, 0.2, 0.0, 0.8, 0.8, 0.0, 0.9, 0.3, 0.3, 0.5, 0.9, 0.9, 0.6, 0.8, 1.0, 0.4, 0.9, 0.7, 0.4, 0.2, 0.8, 0.3, 0.9, 0.2, 0.9, 0.4, 0.8, 0.7, 0.2, 0.3, 0.1, 0.9, 0.1, 0.7, 0.3, 0.9, 0.5, 0.5, 0.9, 0.4, 0.8, 0.9, 0.9, 0.7, 0.0, 0.7, 0.3, 0.1, 0.5, 0.9, 0.2, 0.7, 0.9, 0.7, 0.2, 0.4, 0.1, 0.0, 0.8, 0.9, 0.2, 1.0, 0.9, 0.7, 0.4, 0.9, 0.1, 0.1, 0.1, 0.2, 0.2, 0.9, 0.6, 0.3, 0.9, 0.7, 0.8, 0.3, 0.3, 0.9, 0.9, 0.0, 0.1, 0.7, 0.9, 0.9, 0.8, 0.9, 0.2, 0.7, 0.3, 0.3, 0.3, 0.6, 0.2, 0.1, 0.3, 0.8, 0.8, 0.9, 0.9, 0.9, 0.3, 0.6, 0.2, 0.0, 0.1, 0.3, 0.4, 0.9, 0.9, 0.9, 0.9, 0.4, 0.5, 0.9, 0.4, 0.7, 0.8, 0.9, 0.1, 0.9, 0.1, 0.3, 0.1, 0.8, 0.3, 0.3, 0.3, 0.1, 0.2, 0.3, 0.5, 0.5, 0.9, 0.9, 0.9, 0.9, 0.9, 0.2, 0.8, 0.9, 0.4, 0.3, 0.4, 0.2, 0.2, 0.4, 0.3, 0.8, 0.8, 0.3, 0.2, 0.7, 0.5, 0.1, 0.9, 0.9, 0.2, 0.8, 0.9, 0.3, 0.3, 0.9, 0.4, 0.4, 0.8, 0.4, 0.8, 0.9, 0.2, 0.6, 0.1, 0.3, 0.4, 0.5, 0.3, 0.3, 0.5, 0.1, 0.2, 0.9, 0.4, 0.9, 0.1, 0.9, 0.9, 0.3, 0.9, 0.8, 0.7, 0.7, 0.4, 0.3, 0.1, 0.3, 0.2, 0.3, 0.9, 0.4, 0.2, 0.6, 0.7, 0.1, 0.3, 0.8, 0.8, 0.2, 1.0, 0.1, 0.9, 0.2, 0.4, 0.9, 0.9, 0.7, 0.0, 0.4, 0.5, 0.3, 0.4, 0.2, 0.3, 0.7, 0.6, 0.9, 0.9, 1.0, 0.3, 1.0, 0.9, 0.3, 0.6, 0.4, 0.9, 0.1, 0.2, 0.1, 0.5, 0.8, 0.7, 0.3, 0.9, 0.9, 0.3, 0.2, 0.3, 0.6, 0.3, 0.9, 0.8, 0.9, 0.9, 0.3, 0.3, 0.7, 0.2, 0.1, 0.8, 0.9, 0.7, 0.9, 0.3, 0.9, 0.8, 0.9, 0.3, 0.1, 0.5, 0.3, 0.3, 0.8, 0.3, 0.8, 0.2, 0.2, 0.9, 0.2, 0.9, 0.9, 0.9, 0.3, 0.8, 0.6, 0.6, 0.8, 0.7, 0.2, 0.9, 0.9, 0.9, 1.0, 0.9, 0.1, 0.8, 0.9, 0.9, 0.9, 0.7, 0.9, 0.9, 0.3, 0.5, 0.8, 0.5, 0.3, 0.2, 0.2, 0.3, 0.2, 0.3, 0.2, 0.2, 0.6, 0.1, 0.9, 0.3, 0.7, 0.9, 0.4, 0.6, 0.7, 0.9, 0.1, 0.9, 0.6, 0.8, 0.2, 0.0, 0.2, 0.2, 0.3, 0.8, 0.8, 0.2, 0.2, 0.1, 0.2, 0.7, 0.8, 0.5, 0.4, 0.9, 0.1, 0.4, 0.4, 0.1, 0.9, 0.8, 0.9, 0.9, 0.9, 0.9, 0.3, 0.5, 0.4, 0.3, 0.3, 0.8, 0.9, 0.3, 0.8, 0.4, 0.5, 0.2, 0.2, 0.7, 0.7, 0.9, 0.3, 0.8, 0.5, 0.2, 0.4, 0.9, 0.4, 0.8, 0.7, 0.4, 0.3, 0.1, 0.1, 0.9, 0.3, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8, 0.4, 0.8, 0.8, 0.7, 0.9, 0.2, 0.7, 0.2, 0.4, 0.4, 0.9, 0.2, 0.1, 0.9, 0.4, 0.9, 0.9, 0.3, 0.9, 0.2, 0.5, 0.8, 0.3, 0.9, 0.8, 0.4, 0.6, 0.9, 0.8, 0.3, 1.0, 0.4, 0.4, 0.3, 0.9, 0.1, 0.3, 0.4, 0.3, 0.3, 0.6, 0.3, 0.3, 0.9, 0.9, 0.3, 0.9, 0.4, 0.5, 0.2, 0.0, 0.2, 0.1, 0.8, 0.9, 0.9, 0.1, 0.4, 0.6, 0.5, 0.4, 0.9, 0.2, 0.9, 0.7, 0.5, 0.8, 0.8, 0.8, 0.5, 0.1, 0.1, 0.9, 0.2, 0.0, 0.7, 0.8, 0.9, 0.7, 0.7, 0.9, 0.3, 0.9, 0.9, 0.4, 0.5, 0.7, 0.9, 0.5, 0.9, 0.9, 0.5, 0.9, 0.4, 0.3, 0.3, 0.6, 0.4, 0.6, 0.4, 0.2, 0.2, 0.2, 0.9, 0.8, 0.5, 0.3, 0.2, 0.1, 1.0, 0.9, 0.1, 0.8, 0.7, 0.9, 0.3, 0.3, 0.9, 0.3, 0.2, 0.1, 0.2, 0.8, 0.9, 0.4, 0.7, 0.3, 0.5, 0.2, 0.3, 0.9, 0.8, 0.4, 0.8, 0.4, 0.1, 0.3, 0.3, 0.9, 0.6, 0.2, 0.5, 0.2, 0.2, 0.4, 1.0, 0.5, 0.7, 0.4, 0.3, 0.8, 0.6, 0.4, 0.2, 0.2, 0.5, 0.2, 0.8, 0.9, 0.5, 0.4, 0.4, 0.5, 0.3, 0.8, 0.9, 0.9, 0.9, 0.2, 0.4, 0.1, 0.9, 0.9, 0.9, 0.9, 0.6, 0.9, 0.9, 0.2, 0.2, 0.1, 0.5, 0.1, 0.7, 0.7, 0.8, 0.9, 0.4, 0.3, 0.5, 0.6, 0.5, 0.9, 0.9, 1.0, 0.8, 0.9, 0.5, 0.2, 0.3, 0.9, 0.5, 0.9, 0.2, 0.8, 0.1, 0.1, 0.9, 0.9, 0.3, 0.5, 0.9, 0.4, 0.7, 0.5, 0.9, 0.0, 0.9, 0.8, 0.2, 0.4, 0.6, 0.9, 0.7, 0.4, 0.7, 0.6, 0.3, 0.8, 0.9, 0.2, 0.9, 0.8, 0.8, 0.2, 0.2, 0.1, 0.9, 0.6, 0.5, 0.3, 0.2, 0.9, 0.8, 0.7, 0.2, 1.0, 0.3, 0.2, 0.2, 0.4, 0.2, 0.9, 0.4, 0.1, 0.8, 0.8, 0.8, 0.2, 0.8, 0.7, 0.5, 0.2, 0.4, 0.9, 0.3, 0.9, 0.1, 0.9, 0.8, 0.9, 0.7, 0.9, 1.0, 0.9, 0.2, 0.9, 0.3, 0.7, 0.8, 1.0, 0.2, 0.2, 0.7, 0.7, 0.5, 0.7, 0.6, 0.9, 0.9, 0.8, 0.3, 0.9, 0.9, 0.6, 0.8, 0.9, 0.8, 0.9, 0.9, 0.6, 0.8, 0.3, 0.9, 0.1, 0.1, 0.3, 0.9, 1.0, 0.7, 0.3, 0.3, 0.3, 0.1, 0.7, 0.5, 0.9, 0.6, 0.1, 0.8, 0.8, 0.7, 0.3, 0.2, 0.8, 0.4, 0.7, 0.6, 0.8, 0.1, 0.7, 0.8, 0.9, 1.0, 0.5, 0.6, 0.2, 0.8, 0.9, 0.1, 0.0, 0.3, 0.9, 0.9, 0.7, 0.5, 0.9, 0.0, 0.9, 0.9, 0.9, 0.9, 0.8, 0.7, 0.9, 0.6, 0.8, 0.9, 0.1, 0.1, 0.3, 0.9, 0.2, 0.9, 1.0, 0.9, 0.9, 0.7, 0.7, 0.9, 0.7, 0.9, 0.9, 0.5, 0.2, 0.1, 0.0, 0.9, 0.3, 0.5, 0.9, 0.6, 0.9, 0.2, 0.8, 0.9, 0.1, 0.3, 0.5, 0.8, 0.8, 0.3, 1.0, 0.6, 0.9, 0.8, 0.4, 0.9, 0.9, 0.1, 0.2, 0.1, 0.6, 0.9, 0.9, 0.2, 0.6, 0.2, 0.0, 0.3, 0.9, 0.0, 0.2, 0.6, 0.8, 0.8, 0.1, 0.8, 0.3, 0.1, 0.1, 0.2, 0.9, 0.9, 1.0, 0.4, 0.5, 0.9, 0.9, 0.8, 0.1, 0.1, 0.9, 0.1, 0.9, 0.0, 0.2, 0.7, 0.9, 0.8, 0.9, 0.5, 0.7, 0.1, 0.1, 0.1, 0.9, 0.6, 0.2, 0.7, 0.9, 0.4, 1.0, 0.7, 0.7, 0.5, 0.9, 0.1, 0.7, 0.9, 0.6, 0.5, 0.3, 0.7, 0.4, 0.9, 0.3, 0.2, 0.3, 0.2, 0.5, 0.9, 0.5, 0.2, 0.3, 0.1, 0.1, 0.3, 0.7, 0.5, 0.2, 0.5, 0.9, 0.4, 0.2, 0.3, 0.3, 0.1, 0.2, 0.5, 0.5, 0.8, 0.5, 0.3, 0.3, 0.6, 0.7, 0.9, 0.9, 0.2, 0.3, 0.8, 0.9, 0.5, 0.2, 0.2, 0.7, 0.3, 0.9, 0.3, 0.8, 0.2, 0.2, 0.9, 0.7, 0.2, 0.9, 0.2, 0.8, 0.9, 0.9, 0.9, 0.1, 0.2, 0.6, 0.8, 0.2, 0.3, 0.5, 0.9, 0.2, 0.2, 0.8, 0.3, 0.9, 0.2, 0.9, 0.2, 0.7, 0.4, 0.5, 0.9, 0.5, 0.8, 0.4, 0.5, 0.5, 0.9, 0.4, 0.3, 0.8, 0.8, 0.6, 0.6, 0.8, 0.5, 0.4, 0.8, 0.9, 0.3, 0.3, 1.0, 0.9, 1.0, 0.9, 0.5, 0.8, 0.5, 0.6, 0.9, 0.8, 0.1, 0.9, 0.9, 0.5, 0.9, 0.9, 0.4, 0.2, 0.9, 0.3, 0.3, 0.6, 0.9, 0.9, 0.7, 0.5, 0.2, 0.6, 0.9, 0.9, 0.4, 0.2, 0.7, 0.9, 0.2, 0.5, 0.9, 0.8, 0.9, 0.4, 0.9, 0.9, 0.9, 0.9, 0.3, 0.2, 0.2, 0.9, 0.9, 0.9, 0.9, 0.1, 0.4, 0.5, 0.9, 0.3, 0.9, 0.8, 0.4, 0.5, 0.6, 0.3, 0.8, 0.3, 0.2, 0.9, 0.8, 0.2, 0.9, 0.8, 0.5, 0.9, 0.0, 0.9, 0.9, 0.6, 0.7, 0.3, 0.3, 0.9, 0.9, 0.3, 0.4, 0.9, 0.8, 0.9, 0.1, 0.9, 0.7, 0.4, 0.7, 0.5, 0.3, 0.9, 0.9, 0.3, 0.8, 0.2, 0.9, 0.2, 0.9, 1.0, 0.9, 0.5, 0.8]
sim_preds = [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0]
sim_pers = [86.83, 68.54, 91.67, 84.58, 96.89, 0.48, 30.25, 94.14, 23.72, 93.51, 11.48, 5.44, 60.11, 7.86, 50.56, 71.8, 90.4, 8.14, 89.24, 54.6, 89.92, 25.68, 91.51, 99.97, 26.67, 28.43, 89.84, 88.76, 89.65, 86.49, 28.9, 76.73, 38.37, 87.31, 54.42, 70.37, 95.79, 40.47, 29.76, 65.84, 80.45, 68.75, 93.31, 62.4, 52.02, 63.62, 94.3, 89.14, 93.08, 41.78, 51.84, 10.58, 91.16, 52.49, 50.39, 57.13, 90.15, 10.26, 43.03, 61.76, 87.34, 56.8, 26.37, 3.31, 48.92, 54.44, 46.52, 72.52, 87.91, 84.51, 21.24, 59.18, 45.72, 80.84, 60.9, 89.96, 80.9, 89.59, 49.85, 94.29, 79.42, 84.4, 83.2, 92.47, 23.15, 70.14, 56.52, 9.22, 6.1, 70.02, 69.97, 37.82, 89.16, 33.85, 80.5, 72.93, 30.42, 68.93, 59.5, 88.89, 31.19, 90.55, 44.2, 47.09, 11.71, 59.35, 85.87, 50.52, 86.96, 65.44, 74.8, 58.74, 61.86, 77.32, 93.12, 53.9, 80.98, 56.15, 89.4, 72.94, 42.32, 92.52, 69.66, 73.39, 75.01, 13.99, 79.23, 71.68, 76.08, 60.23, 64.37, 80.74, 34.85, 48.71, 56.46, 79.74, 46.59, 79.44, 35.74, 86.76, 30.43, 89.87, 25.78, 76.87, 44.88, 50.85, 10.97, 15.47, 54.16, 82.34, 44.86, 70.49, 37.54, 77.18, 57.47, 87.45, 21.57, 59.68, 40.75, 20.94, 15.16, 69.73, 52.61, 96.99, 67.22, 85.18, 46.41, 97.6, 38.64, 56.34, 68.14, 47.02, 21.99, 24.17, 89.39, 28.77, 91.18, 35.31, 78.28, 81.68, 33.89, 31.72, 26.9, 97.53, 46.0, 38.17, 36.39, 69.98, 70.62, 87.21, 95.9, 53.35, 38.53, 65.79, 51.75, 66.77, 82.29, 77.81, 64.87, 71.97, 86.5, 74.92, 99.44, 54.82, 13.23, 65.94, 76.92, 45.64, 25.46, 37.32, 64.59, 50.87, 39.98, 73.31, 34.31, 75.16, 60.92, 80.47, 39.55, 76.17, 27.71, 45.78, 21.03, 83.97, 43.82, 82.66, 81.22, 83.54, 81.33, 86.81, 31.31, 43.44, 71.19, 56.86, 87.45, 31.48, 69.11, 42.11, 84.75, 68.27, 82.2, 35.75, 24.54, 35.01, 32.25, 22.6, 41.32, 72.82, 41.09, 81.28, 63.63, 24.26, 83.06, 83.3, 53.44, 19.2, 34.7, 33.59, 81.58, 63.31, 35.87, 15.99, 64.51, 56.17, 36.4, 70.59, 53.13, 40.61, 26.34, 91.76, 64.05, 27.75, 70.47, 86.4, 82.11, 39.94, 21.23, 37.24, 89.78, 76.11, 51.89, 23.11, 30.81, 55.41, 54.26, 89.96, 89.03, 63.08, 78.0, 74.33, 81.92, 35.39, 35.89, 55.64, 85.44, 22.58, 77.31, 62.21, 67.8, 41.16, 86.35, 61.42, 70.67, 32.91, 45.85, 74.83, 83.4, 42.07, 34.19, 93.86, 40.25, 93.03, 16.44, 63.33, 45.2, 38.62, 34.45, 69.91, 50.95, 90.96, 36.79, 70.93, 30.49, 25.82, 26.91, 86.57, 27.42, 55.12, 77.79, 68.17, 72.89, 70.07, 51.65, 37.38, 11.97, 24.26, 75.82, 58.47, 76.16, 73.08, 55.16, 20.68, 45.44, 68.34, 80.89, 87.84, 75.48, 85.6, 30.56, 43.02, 72.06, 82.32, 28.88, 87.95, 55.57, 84.38, 97.51, 80.46, 37.98, 24.49, 75.49, 15.06, 25.89, 28.02, 22.43, 38.64, 32.1, 34.76, 55.52, 74.29, 59.91, 31.01, 55.76, 71.92, 75.15, 93.47, 44.55, 55.46, 55.09, 60.57, 25.56, 66.22, 86.65, 77.29, 69.39, 63.26, 50.13, 21.51, 32.38, 54.15, 79.39, 54.93, 10.43, 86.75, 83.23, 51.9, 83.44, 88.9, 37.76, 65.26, 22.55, 84.4, 40.38, 80.76, 84.18, 80.39, 45.9, 42.95, 90.33, 52.62, 37.97, 31.62, 76.42, 21.12, 91.69, 58.78, 81.15, 30.86, 79.52, 70.27, 64.51, 46.27, 63.59, 25.45, 58.23, 62.84, 90.36, 25.37, 92.69, 8.75, 73.15, 28.77, 54.02, 12.99, 52.13, 34.11, 86.84, 27.92, 91.92, 0.11, 69.83, 15.28, 29.95, 25.68, 63.56, 51.51, 72.0, 56.35, 44.71, 52.58, 11.26, 20.12, 70.44, 24.47, 94.85, 78.16, 78.83, 32.86, 47.23, 48.96, 56.93, 80.2, 35.35, 44.51, 82.92, 31.21, 27.69, 14.98, 25.78, 33.89, 43.84, 54.24, 87.93, 88.5, 50.16, 74.38, 30.21, 33.29, 46.45, 68.19, 52.59, 31.55, 88.54, 9.36, 63.15, 52.21, 82.17, 80.99, 48.94, 35.81, 66.29, 69.64, 60.2, 76.88, 54.82, 36.84, 5.94, 72.65, 4.58, 73.1, 87.55, 88.77, 75.15, 64.38, 81.47, 38.88, 57.23, 7.18, 24.8, 67.86, 1.74, 6.07, 38.76, 63.9, 38.98, 43.29, 55.89, 29.57, 38.43, 79.7, 53.34, 48.68, 82.26, 32.82, 71.92, 0.12, 54.41, 41.51, 55.77, 78.12, 29.43, 44.57, 2.03, 60.58, 7.26, 14.63, 25.99, 76.36, 77.23, 65.51, 89.88, 70.22, 5.77, 59.26, 47.82, 90.4, 40.62, 51.66, 61.35, 8.46, 72.7, 83.29, 29.37, 54.5, 76.7, 66.5, 44.62, 81.31, 1.8, 67.22, 74.33, 80.76, 30.47, 21.14, 78.44, 78.53, 19.94, 3.27, 73.86, 35.95, 37.2, 26.55, 43.29, 66.73, 46.45, 49.41, 59.64, 50.78, 17.08, 80.96, 26.37, 74.39, 71.31, 89.26, 73.16, 25.56, 77.66, 60.21, 63.12, 89.68, 25.15, 34.58, 64.06, 38.42, 79.21, 47.64, 69.96, 73.64, 39.86, 53.3, 8.29, 87.6, 31.4, 57.18, 79.98, 68.0, 76.32, 92.92, 43.74, 31.15, 47.64, 74.43, 0.28, 74.93, 14.74, 80.15, 52.09, 60.68, 68.69, 2.32, 81.43, 23.37, 42.0, 51.82, 81.07, 53.82, 55.93, 57.62, 36.44, 63.03, 80.67, 57.31, 22.55, 83.87, 74.36, 66.52, 73.35, 64.12, 67.85, 10.02, 21.03, 69.59, 38.25, 26.67, 75.24, 31.25, 67.48, 51.87, 29.41, 37.01, 58.11, 46.06, 44.38, 30.19, 11.7, 43.61, 33.76, 58.67, 33.49, 69.27, 63.0, 68.5, 34.66, 66.1, 76.5, 42.9, 21.19, 59.11, 67.24, 12.27, 71.31, 34.04, 27.96, 51.92, 79.4, 82.89, 50.42, 54.75, 93.75, 45.44, 68.58, 48.34, 43.71, 43.15, 67.75, 46.48, 61.51, 30.07, 73.68, 33.57, 57.57, 66.38, 25.92, 35.79, 12.91, 88.69, 24.04, 62.76, 40.66, 75.28, 46.37, 58.93, 84.1, 43.01, 67.15, 86.35, 83.69, 58.01, 4.23, 62.34, 40.05, 20.33, 47.66, 74.89, 29.04, 65.32, 86.74, 61.77, 22.0, 46.74, 26.64, 2.94, 74.99, 86.24, 23.35, 85.75, 98.0, 58.58, 39.59, 87.4, 13.07, 21.76, 11.22, 36.76, 18.47, 90.75, 61.64, 37.75, 78.79, 61.86, 77.38, 50.51, 36.57, 77.26, 73.17, 2.56, 34.81, 51.41, 89.96, 72.58, 54.2, 80.66, 23.8, 64.8, 30.41, 29.5, 29.74, 54.79, 35.08, 17.26, 33.01, 68.59, 64.98, 88.11, 79.63, 90.88, 50.03, 63.22, 35.5, 13.92, 24.0, 34.06, 48.06, 82.44, 81.05, 85.08, 89.19, 43.52, 52.11, 84.69, 52.4, 57.72, 64.47, 80.51, 8.02, 92.14, 20.42, 53.95, 27.8, 81.23, 48.62, 36.07, 29.93, 19.85, 26.44, 31.44, 55.83, 50.58, 84.24, 75.14, 77.66, 79.49, 82.65, 23.6, 58.77, 92.85, 38.21, 56.72, 34.95, 35.33, 30.38, 45.93, 38.04, 53.52, 59.2, 27.44, 32.23, 63.3, 56.2, 7.31, 80.51, 89.72, 31.31, 64.3, 81.45, 42.26, 32.93, 81.69, 36.1, 40.7, 64.85, 32.35, 58.59, 67.01, 40.75, 57.68, 22.84, 37.12, 51.02, 67.33, 35.12, 45.62, 53.28, 25.47, 22.64, 81.64, 41.31, 72.98, 9.0, 81.63, 78.9, 43.11, 77.78, 67.76, 51.52, 49.81, 42.82, 44.19, 7.75, 33.91, 33.63, 60.47, 66.2, 54.79, 44.38, 55.39, 58.99, 15.25, 35.65, 66.58, 71.98, 32.55, 82.27, 21.98, 79.53, 30.69, 34.41, 78.1, 88.39, 70.18, 9.11, 42.04, 49.49, 36.59, 50.15, 31.22, 27.38, 59.0, 58.01, 87.04, 88.07, 92.44, 28.11, 72.82, 80.09, 39.0, 49.97, 60.96, 74.93, 18.6, 31.22, 13.91, 59.66, 68.87, 61.34, 36.12, 62.0, 79.23, 31.59, 21.92, 31.56, 53.83, 39.0, 83.6, 73.01, 76.77, 84.86, 49.33, 48.48, 59.35, 44.15, 15.28, 60.52, 66.02, 53.84, 83.67, 30.48, 78.33, 70.21, 79.69, 41.25, 21.25, 53.97, 46.64, 28.11, 62.28, 38.15, 69.18, 21.83, 33.53, 58.09, 25.54, 72.24, 90.65, 73.03, 27.75, 56.78, 59.85, 51.93, 56.12, 55.35, 32.7, 81.21, 63.67, 80.45, 92.02, 71.5, 32.22, 62.05, 85.28, 71.71, 92.94, 66.22, 80.42, 82.12, 39.15, 39.37, 76.15, 57.44, 31.74, 34.52, 21.6, 46.3, 17.63, 37.82, 26.77, 23.58, 68.13, 11.92, 89.49, 31.62, 68.94, 83.76, 47.93, 52.28, 55.49, 70.58, 28.0, 69.81, 61.36, 71.63, 22.29, 15.01, 30.14, 20.29, 30.81, 59.16, 73.19, 46.0, 13.7, 12.17, 25.58, 56.66, 81.76, 69.57, 52.3, 79.33, 24.96, 49.7, 45.18, 30.7, 85.94, 66.77, 83.48, 81.64, 64.27, 78.36, 43.55, 53.51, 47.95, 38.08, 46.95, 66.84, 87.26, 32.19, 59.42, 40.79, 62.65, 34.5, 41.85, 62.89, 55.81, 60.47, 32.63, 68.5, 59.35, 25.65, 47.42, 73.52, 36.49, 57.86, 63.46, 53.78, 35.83, 14.85, 16.16, 92.86, 34.31, 76.69, 79.43, 84.87, 67.35, 72.79, 64.16, 49.31, 68.29, 58.62, 65.11, 72.03, 22.37, 61.64, 16.0, 42.74, 54.48, 77.81, 19.82, 33.41, 75.93, 32.99, 84.03, 85.6, 32.66, 69.78, 32.45, 51.25, 53.49, 32.51, 86.88, 70.1, 40.56, 62.5, 88.27, 55.69, 29.69, 88.93, 34.83, 48.11, 48.1, 73.24, 11.21, 39.05, 57.54, 42.23, 37.97, 54.72, 30.24, 34.06, 67.3, 65.81, 31.7, 76.64, 41.62, 50.02, 40.36, 6.3, 24.12, 11.75, 73.81, 71.7, 78.51, 13.7, 49.72, 66.28, 44.04, 48.6, 77.38, 40.4, 78.76, 63.92, 49.43, 70.0, 60.84, 67.29, 46.61, 13.52, 40.29, 72.36, 38.96, 6.27, 61.6, 69.98, 81.88, 49.97, 58.23, 85.82, 43.2, 74.29, 79.77, 55.82, 55.54, 62.53, 71.93, 53.97, 77.14, 87.21, 46.75, 78.1, 43.78, 32.12, 30.64, 55.31, 54.29, 58.39, 55.19, 26.17, 11.33, 28.44, 89.48, 51.23, 44.38, 49.07, 26.9, 32.16, 92.41, 86.15, 24.75, 56.59, 54.64, 65.0, 47.96, 51.27, 83.88, 47.58, 17.12, 20.13, 36.36, 73.45, 67.28, 38.35, 58.75, 50.18, 59.37, 34.33, 39.89, 68.93, 60.15, 44.1, 71.09, 50.65, 29.09, 37.46, 36.92, 84.53, 64.46, 39.63, 52.48, 30.39, 22.32, 41.72, 91.52, 57.94, 68.26, 41.11, 46.43, 72.0, 43.99, 44.37, 18.61, 36.74, 40.09, 42.76, 74.39, 78.94, 44.69, 61.49, 43.96, 44.44, 40.92, 72.16, 67.12, 66.28, 74.77, 23.15, 54.82, 17.12, 66.84, 88.62, 79.52, 76.97, 53.73, 65.97, 74.01, 36.67, 30.94, 22.33, 56.14, 27.89, 60.97, 63.75, 63.85, 70.23, 42.58, 29.41, 61.57, 58.41, 42.52, 79.92, 79.97, 91.27, 71.86, 76.2, 44.22, 48.26, 38.05, 76.69, 54.81, 82.92, 24.59, 65.16, 20.43, 21.37, 76.2, 84.77, 55.14, 47.81, 89.89, 32.09, 49.25, 50.54, 69.44, 3.9, 74.72, 64.96, 41.3, 56.93, 59.71, 71.48, 64.07, 44.56, 52.34, 59.22, 28.47, 66.97, 68.9, 42.18, 83.6, 69.62, 68.36, 22.37, 26.6, 25.73, 73.79, 53.88, 49.38, 55.8, 43.38, 71.92, 75.69, 67.41, 21.14, 90.61, 47.15, 27.66, 36.26, 51.94, 43.76, 75.78, 35.63, 32.76, 48.18, 75.23, 68.63, 37.62, 53.18, 62.7, 53.01, 25.66, 43.56, 77.44, 56.79, 63.74, 20.89, 65.7, 68.1, 61.32, 63.69, 74.59, 82.25, 80.06, 24.73, 86.72, 36.09, 55.46, 62.64, 90.34, 14.12, 35.46, 54.47, 54.77, 52.05, 42.98, 64.8, 84.8, 91.12, 67.22, 40.08, 61.73, 81.53, 60.14, 52.86, 73.42, 59.44, 54.87, 90.47, 62.71, 71.68, 42.85, 62.04, 16.87, 26.26, 35.89, 84.79, 75.25, 51.9, 39.64, 46.59, 36.37, 13.73, 69.69, 55.02, 76.84, 54.45, 18.38, 57.84, 60.46, 58.53, 34.18, 29.47, 68.01, 46.0, 59.16, 53.59, 58.19, 10.18, 60.62, 68.53, 86.57, 92.45, 60.15, 68.59, 27.05, 63.07, 73.05, 33.85, 6.55, 39.39, 83.09, 65.95, 64.01, 57.92, 79.64, 27.64, 90.74, 74.35, 71.66, 81.08, 59.0, 73.47, 92.9, 54.99, 74.63, 74.33, 15.08, 34.95, 31.82, 67.33, 40.85, 87.72, 85.89, 83.48, 74.46, 64.62, 60.61, 73.65, 65.7, 63.58, 70.93, 51.22, 20.97, 19.49, 5.24, 85.98, 35.12, 50.93, 78.29, 60.36, 73.68, 20.03, 61.37, 83.66, 17.69, 48.73, 51.82, 66.92, 62.97, 39.6, 92.54, 62.73, 71.07, 63.53, 45.03, 92.36, 83.4, 21.85, 31.53, 15.98, 61.83, 86.54, 82.55, 21.34, 70.25, 23.7, 22.22, 39.56, 81.19, 4.47, 18.84, 59.92, 67.03, 57.1, 14.67, 73.98, 47.6, 32.6, 14.28, 38.97, 83.56, 90.54, 86.94, 35.03, 52.95, 60.75, 83.76, 59.68, 8.13, 10.63, 84.82, 18.12, 63.78, 13.72, 37.64, 56.85, 87.26, 61.92, 83.67, 52.76, 68.91, 30.68, 9.34, 17.78, 85.66, 57.29, 32.17, 65.62, 84.38, 43.06, 93.6, 60.83, 62.95, 50.59, 55.76, 19.36, 63.4, 73.15, 46.31, 57.19, 37.16, 56.08, 43.0, 84.51, 42.68, 43.64, 32.21, 29.54, 50.96, 80.54, 51.79, 29.24, 50.95, 14.36, 20.51, 42.31, 65.41, 47.33, 40.7, 45.2, 63.46, 46.8, 35.03, 42.89, 42.28, 17.76, 29.73, 47.58, 52.21, 70.24, 57.63, 35.13, 39.09, 60.57, 57.01, 81.71, 90.7, 28.09, 30.94, 70.71, 83.32, 59.36, 40.6, 22.29, 55.28, 30.92, 72.02, 43.75, 73.06, 29.8, 49.57, 76.84, 71.04, 24.68, 83.83, 40.02, 61.53, 78.53, 87.22, 77.63, 22.84, 35.18, 56.4, 59.45, 24.4, 30.25, 63.37, 86.42, 49.59, 30.71, 72.6, 44.89, 78.07, 36.7, 83.41, 11.99, 60.24, 60.55, 45.46, 70.06, 60.12, 63.23, 39.21, 49.31, 46.87, 80.78, 45.61, 36.51, 71.94, 61.57, 54.39, 55.27, 69.39, 48.98, 41.2, 71.59, 87.31, 51.65, 32.48, 68.96, 68.6, 94.26, 70.25, 54.61, 74.57, 52.84, 50.19, 88.08, 70.34, 31.8, 80.89, 70.26, 49.41, 90.67, 74.8, 49.48, 24.43, 74.76, 32.19, 43.6, 52.7, 85.92, 81.99, 65.99, 52.53, 44.5, 58.63, 77.85, 88.87, 37.33, 21.33, 61.49, 84.43, 29.16, 51.0, 87.15, 56.22, 63.99, 47.36, 81.42, 84.6, 69.19, 86.61, 33.35, 44.56, 21.37, 86.26, 82.86, 88.78, 78.15, 31.92, 57.03, 56.74, 67.65, 38.93, 86.07, 67.15, 33.72, 50.58, 58.27, 33.53, 72.67, 32.5, 34.18, 87.2, 71.5, 35.34, 71.42, 64.48, 53.18, 78.42, 22.43, 85.7, 63.91, 55.37, 50.95, 43.32, 31.29, 80.78, 90.91, 48.89, 42.56, 80.12, 71.06, 89.92, 23.93, 85.97, 60.11, 37.3, 51.79, 59.41, 26.84, 83.68, 78.37, 50.26, 55.87, 27.0, 82.45, 22.68, 90.02, 81.56, 77.07, 47.35, 53.48]

threshold = 85
sim_preds = [1 if per >= threshold else 0 for per in sim_pers]
print(100*sum(sim_preds)/len(sim_preds))

threshold2 = 0
det_preds = [1 if per >= threshold2 else 0 for per in det_pers]
print(100*sum(det_preds)/len(det_preds))

9.883720930232558
100.0


In [153]:
from sklearn.metrics import classification_report

def display_report(report, title, display_extras=None):
    separator_char_width = 55
    bullet_point = " - "
    primary_sep = "="
    secondary_sep = "-"
    print()
    print(primary_sep*separator_char_width)
    print(title)
    print(primary_sep*separator_char_width)
    if display_extras:
        for extra in display_extras:
            print(bullet_point + extra)
    print(bullet_point + f"Accuracy:  {100 * report['accuracy']:.2f}%")
    class_labels = [key for key in report.keys() if key != 'accuracy']
    for class_label in class_labels:
        print(secondary_sep*separator_char_width)
        print(bullet_point + f"Class {class_label} ({int(report[class_label]['support'])} tot):")
        print(f"\tRecall:    {report[class_label]['recall']:.4f}")
        print(f"\tPrecision: {report[class_label]['precision']:.4f}")
        print(f"\tF1-score:  {report[class_label]['f1-score']:.4f}")
    print(primary_sep*separator_char_width)

y_test = pd.Series(y_test)
det_right_sim_wrong = [1 if d != s and d == t else 0 for d, s, t in zip(det_preds, sim_preds, y_test)]
det_target_names = list(set(list(y_test) + list(det_preds)))
det_report_extras = [f"Deterministic-right-simulation-wrong number: {sum(det_right_sim_wrong)}"]
det_report = classification_report(y_test, det_preds, target_names=det_target_names, output_dict=True)
display_report(det_report, "Deterministic Report", display_extras=det_report_extras)

sim_right_det_wrong = [1 if d != s and s == t else 0 for d, s, t in zip(det_preds, sim_preds, y_test)]
sim_target_names = list(set(list(y_test) + list(sim_preds)))
sim_report_extras = [f"Simulation-right-deterministic-wrong number: {sum(sim_right_det_wrong)}"]
sim_report = classification_report(y_test, sim_preds, target_names=sim_target_names, output_dict=True)
display_report(sim_report, "Simulation Report", display_extras=sim_report_extras)

det_sim_agree = [1 if d == s else 0 for d, s, t in zip(det_preds, sim_preds, y_test)]
if len(det_sim_agree) > 0:
    y_test_agree = [y for y, a in zip(y_test, det_sim_agree) if a == 1]
    agree_preds = [p for p, a in zip(det_preds, det_sim_agree) if a == 1]
    agree_target_names = list(set(list(y_test_agree) + list(agree_preds)))
    agree_report_extras = [f"Deterministic-and-simulation-agree number: {len(agree_preds)} ({(100 * len(agree_preds)/len(det_preds)):.2f}%)"]
    agree_report = classification_report(y_test_agree, agree_preds, target_names=agree_target_names, output_dict=True)
    display_report(agree_report, "Agree Report", display_extras=agree_report_extras)

det_sim_disagree = [i for i, (d, s) in enumerate(zip(det_preds, sim_preds)) if d != s]
if len(det_sim_disagree) > 0:
    y_test_disagree = [y_test.iloc[i] for i in det_sim_disagree]
    disagree_report_extras = [f"Disagree number: {len(det_sim_disagree)} ({(100 * len(det_sim_disagree)/len(det_preds)):.2f}%)"]
    
    det_disagree_preds = [det_preds[i] for i in det_sim_disagree]
    det_target_names = list(set(list(y_test_disagree) + list(det_disagree_preds)))
    det_disagree_report = classification_report(y_test_disagree, det_disagree_preds, target_names=det_target_names, output_dict=True)
    display_report(det_disagree_report, "Deterministic Disagree Report", display_extras=disagree_report_extras)
    
    sim_disagree_preds = [sim_preds[i] for i in det_sim_disagree]
    sim_target_names = list(set(list(y_test_disagree) + list(sim_disagree_preds)))
    sim_disagree_report = classification_report(y_test_disagree, sim_disagree_preds, target_names=sim_target_names, output_dict=True)
    display_report(sim_disagree_report, "Simulation Disagree Report", display_extras=disagree_report_extras)


Deterministic Report
 - Deterministic-right-simulation-wrong number: 785
 - Accuracy:  54.24%
-------------------------------------------------------
 - Class 0 (787 tot):
	Recall:    0.0000
	Precision: 0.0000
	F1-score:  0.0000
-------------------------------------------------------
 - Class 1 (933 tot):
	Recall:    1.0000
	Precision: 0.5424
	F1-score:  0.7034
-------------------------------------------------------
 - Class macro avg (1720 tot):
	Recall:    0.5000
	Precision: 0.2712
	F1-score:  0.3517
-------------------------------------------------------
 - Class weighted avg (1720 tot):
	Recall:    0.5424
	Precision: 0.2942
	F1-score:  0.3815

Simulation Report
 - Simulation-right-deterministic-wrong number: 765
 - Accuracy:  53.08%
-------------------------------------------------------
 - Class 0 (787 tot):
	Recall:    0.9720
	Precision: 0.4935
	F1-score:  0.6547
-------------------------------------------------------
 - Class 1 (933 tot):
	Recall:    0.1586
	Precision: 0.8706
	

C:\Users\gthra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gthra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gthra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\gthra\anaconda3\lib\site-packag

In [231]:
def get_acc_per_thresholds(for_home_team, sim_pers, y, verbose=True):
    def get_nearest_acc(acc):
        return int((acc // 10) * 10)

    default_acc_per_thresholds_tuple = (0, 101, 0, 0)
    acc_per_thresholds = {
        10 * i: default_acc_per_thresholds_tuple for i in range (11)
    }
    thresholds = np.arange(0, 100, 0.01)
    sim_pers = np.array(sim_pers)
    y = np.array(y)

    if not for_home_team:
        sim_pers = 100 - sim_pers

    for threshold in thresholds:
        threshold = np.round(threshold, 2)
        sub_y = y[sim_pers >= threshold]

        if len(sub_y) > 0:
            accuracy = 0 if len(sub_y) == 0 else np.round(100 * np.count_nonzero(sub_y == int(for_home_team))/len(sub_y), 2)
            nearest_acc = get_nearest_acc(accuracy)
            per_games = np.round((100 * len(sub_y)/len(sim_pers)), 2)

            if for_home_team:
                if accuracy >= nearest_acc and threshold < acc_per_thresholds[nearest_acc][1]:
                    acc_per_thresholds[nearest_acc] = (accuracy, threshold, per_games, len(sub_y))
            else:
                if accuracy >= nearest_acc and threshold < acc_per_thresholds[nearest_acc][1]:
                    acc_per_thresholds[nearest_acc] = (accuracy, threshold, per_games, len(sub_y))

            if verbose:
                print(f"Minimum simulation confidence threshold: {threshold}")
                print(f"Number of games that meet simulation threshold: {len(sub_y)} ({per_games}%)")
                print(f"Accuracy for those games:  {accuracy}%")
                print("="*100)

    key_value_pairs = list(acc_per_thresholds.items())
    for key, value in key_value_pairs:
        if value == default_acc_per_thresholds_tuple or key < 50:
            del acc_per_thresholds[key]

    return acc_per_thresholds

acc_per_thresholds_home = get_acc_per_thresholds(True, sim_pers, y_test, verbose=False)
print(acc_per_thresholds_home)
acc_per_thresholds_away = get_acc_per_thresholds(False, sim_pers, y_test, verbose=False)
print(acc_per_thresholds_away)

{50: (54.24, 0.0, 100.0, 1720), 60: (60.03, 32.18, 80.58, 1386), 70: (70.06, 55.13, 51.45, 885), 80: (80.0, 72.66, 26.74, 460), 90: (90.37, 86.77, 7.85, 135), 100: (100.0, 92.91, 1.51, 26)}
{50: (50.0, 16.34, 88.26, 1518), 60: (60.0, 39.04, 57.27, 985), 70: (70.0, 64.12, 24.42, 420), 80: (80.56, 91.01, 2.09, 36), 90: (90.48, 93.91, 1.22, 21)}


In [232]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb

X_test = X.copy()

det_preds = np.array(det_preds)
sim_preds = np.array(sim_preds)
y_test = np.array(y_test)
det_pers = np.array(det_pers)
sim_pers = np.array(sim_pers)


disagree_indices = np.where(det_preds != sim_preds)[0]


y_disagree = y_test[disagree_indices]
sim_disagree = sim_preds[disagree_indices]
sim_correct = (sim_disagree == y_disagree).astype(int)


meta_features = np.column_stack((
    det_preds[disagree_indices],
    det_pers[disagree_indices],
    sim_preds[disagree_indices],
    sim_pers[disagree_indices]
))


X_disagree = X_test.iloc[disagree_indices].reset_index(drop=True)
meta_df = pd.DataFrame(meta_features, columns=['det_pred', 'det_prob', 'sim_pred', 'sim_prob'])
meta_df = pd.concat([meta_df, X_disagree.reset_index(drop=True)], axis=1)


meta_df['prob_diff_abs'] = (meta_df['det_prob'] - meta_df['sim_prob']).abs()
meta_df['prob_product'] = meta_df['det_prob'] * meta_df['sim_prob']
meta_df['det_more_confident'] = (meta_df['det_prob'] > meta_df['sim_prob']).astype(int)


for col in meta_df.columns:
    if not pd.api.types.is_numeric_dtype(meta_df[col]):
        meta_df[col] = pd.to_numeric(meta_df[col], errors='coerce')


X1 = meta_df.values
y1 = sim_correct

skf = StratifiedKFold(n_splits=5, shuffle=False)

accuracies = []

for train_idx, test_idx in skf.split(X1, y1):
    X_train, X_test_cv = X1[train_idx], X1[test_idx]
    y_train, y_test_cv = y1[train_idx], y1[test_idx]

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test_cv, label=y_test_cv, reference=train_data)

    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.1,
        'verbose': -1,
        'seed': 42,
    }

    model = lgb.train(
        params,
        train_data,
        num_boost_round=200,
        valid_sets=[train_data, valid_data],
    )

    preds_proba = model.predict(X_test_cv, num_iteration=model.best_iteration)
    preds = (preds_proba >= 0.5).astype(int)
    acc = accuracy_score(y_test_cv, preds)
    accuracies.append(acc)

print(f"Meta-model accuracy over 5 folds: {np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}")

Meta-model accuracy over 5 folds: 0.682 ± 0.014
